In [ ]:
# stl-SSNP-08-write-XYZ-file.py
# 2021-12-10
#
# read dem-all-7.csv file
#
import os
path = os.getcwd()
print('當前工作目錄 ==>', path)
#
from stl import mesh
import numpy as np
#
path_data  = './data/'
path_3Dstl = './3Dstl/'
#
## =======================================================================
#
# 台灣橫麥卡托二度分帶投影座標系統（TM2）
#
float_To_int = np.vectorize(np.int32)
#
data = np.loadtxt(path_data + 'dem-all-7.csv',delimiter=',',dtype='float')
min_item = np.amin(data, axis=0) # axis=0 --> min of each column
max_item = np.amax(data, axis=0) # axis=0 --> max of each column
print ('Left Down point:', float_To_int(min_item[0]), float_To_int(min_item[1]))
print ('Right Up point: ' , float_To_int(max_item[0]), float_To_int(max_item[1]))
print ('height (min, max)', min_item[2], max_item[2])
print ('\n')
#
height_min = min_item[2]
height_max = max_item[2]
#
# Left Down point:    0, 2040
# Right Up point:  3800, 9000
# height (min, max) 0.0, 357.21
#
TM2_X = float_To_int(data[..., 0]) # 二度分帶 X座標
TM2_Y = float_To_int(data[..., 1]) # 二度分帶 Y座標
TM2_Z = data[..., 2]               # DEM file z value, float type
#
## -------------------------------------------------------------------------
#
# XYlist is sorted (x, y), first order is y, then x
#
XYlist = list(zip(TM2_X,TM2_Y))
# X, Y value
X = np.arange(-100, 3820, 20)
Y = np.arange(2040, 9020, 20)
Xmesh, Ymesh = np.meshgrid(X, Y)  # x-y 平面的網格
#
# X -100 .. 3800
# Y 2040 .. 9000
# 6980/20 ==> 349
# 3920/20 ==> 196
#
Zmesh = np.zeros(shape=(349,196),dtype=np.float16)
i = 0 ; j = 0
#
for y in np.arange(2040, 9020, 20) :
    print('y is', y)
    z = np.zeros(shape=(196),dtype=np.float16)
    zi = 0
    for x in np.arange(-100, 3820, 20):
        if (x,y) in XYlist :
            # print(str(zi), list([x,y]), TM2_Z[i])
            z[zi] = TM2_Z[i]
            i = i + 1
        else :
            # print(str(zi))
            z[zi] = 0.0
        # end if
        zi = zi + 1
    #end for
    # pprint.pprint(z)
    Zmesh[j] = z
    j = j + 1
#end for
#
factor_3D = 0.01
Xmesh_3D = np.around(factor_3D * Xmesh, 1)
Ymesh_3D = np.around(factor_3D * Ymesh, 1)
Zmesh_3D = np.around(factor_3D * Zmesh, 3)
#
XYZ_list = np.stack([Xmesh_3D, Ymesh_3D, Zmesh_3D], axis=2)
print ('\n')
print('=================')
print(str(np.shape(XYZ_list)))  # shape is ---> (349, 196, 3)
#
min_item = np.amin(XYZ_list, axis=0) # axis=0 --> min of each column
max_item = np.amax(XYZ_list, axis=0) # axis=0 --> max of each column
print ('Left Down point:', float_To_int(min_item[0][0]), float_To_int(min_item[0][1]))
print ('Right Up point: ' , float_To_int(max_item[195][0]), float_To_int(max_item[195][1]))
print ('height (min, max)', factor_3D*height_min, factor_3D*height_max)
print ('\n')
#
# XYZ_list --> shape=(349,196,3)
# create dem_rows
#
dem_rows = []
for i in range(0,349) :
    dem_rows.append([list(xyz) for xyz in XYZ_list[i]])  
# end for
#
# adjust to not show the hole
for i in (343,344,345,346,347,348) :
    dem_rows[i] = [[xyz[0],xyz[1],0.0 ] for xyz in XYZ_list[i]]
# end for
#
# XYZ_list --> shape=(349,196,3)
# adjust to not show the hole
for i in range(300,343,1) :
    j = 0
    for xyz in XYZ_list[i]:
        if j > 190 :
            dem_rows[i][j] = [xyz[0],xyz[1],0.0]
        # end if
        j = j + 1
    # end for
# end for
#
# dem_rows --> shape=(349,196,3)
#
